# Model definition

In [2]:
import pennylane as qml
from pennylane import qchem
from jax import numpy as jnp
import numpy as np
# system setup 
active_electrons = 2 #12
active_orbitals = 3  #8
# atomic symbols defining the molecule
symbols = ['O', 'O']
r = 2.30 # Coordinates in Bohr 
coordinates = jnp.array([[0.0, 0.0, 0.0], [0.0, 0.0, r]])

In [ ]:
# SKIP ME
import optax
from pennylane import AllSinglesDoubles
# Construct the Molecule object
molecule = qchem.Molecule(symbols, coordinates)
# Obtain the qubit Hamiltonian
H, qubits = qchem.molecular_hamiltonian(molecule, active_electrons=active_electrons, active_orbitals=active_orbitals, method='pyscf')
singles, doubles = qchem.excitations(active_electrons, qubits)
hf = qchem.hf_state(active_electrons, qubits)
print(f"Total number of excitations = {len(singles) + len(doubles)}")
# VQE define the device, optimizer and circuit
dev = qml.device("lightning.qubit", wires=qubits)
opt = optax.sgd(learning_rate=0.4) # sgd stands for StochasticGradientDescent

@qml.qnode(dev, interface='jax')
def circuit(parameters):
    AllSinglesDoubles(parameters, range(qubits), hf, singles, doubles)
    return qml.expval(H)  # we are interested in minimizing this expectation value

# initialize the gate parameters
init_params = jnp.zeros(len(singles) + len(doubles))

prev_energy = 0.0
@qml.qjit
def update_step(i, params, opt_state):
    """Perform a single gradient update step"""
    grads = qml.grad(circuit)(params)
    updates, opt_state = opt.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    return (params, opt_state)

opt_state = opt.init(init_params)
params = init_params
energy = 0.0
prev_energy = energy
for i in range(50):
    print (i,params,energy)    
    params, opt_state = update_step(i, params, opt_state)
    energy = circuit(params)
    if jnp.abs(energy - prev_energy) < 1e-6:
        break
    prev_energy = energy


# Ground state optimisation

In [3]:
# Construct the Molecule object
molecule = qchem.Molecule(symbols, coordinates)
# Obtain the qubit Hamiltonian
H, qubits = qchem.molecular_hamiltonian(molecule, active_electrons=active_electrons, active_orbitals=active_orbitals, method='openfermion')

In [17]:
print (len(H),qubits)

34 6


In [5]:
#Initial state preparation and excitations
from pennylane import AllSinglesDoubles
singles, doubles = qchem.excitations(active_electrons, qubits)#, delta_sz = 0)
hf = qchem.hf_state(active_electrons, qubits)
print (hf)
#We set the trial state to the triplet state
#init_state = np.array([1,0,1,0,0,0])
init_state = hf
print (singles)
print (doubles)
print(f"Total number of excitations = {len(singles) + len(doubles)}")

dev = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev, interface='jax')
def circuit(parameters):
    AllSinglesDoubles(parameters, range(qubits), init_state, singles, doubles)
    return qml.expval(H)  # we are interested in minimizing this expectation value

[1 1 0 0 0 0]
[[0, 2], [0, 4], [1, 3], [1, 5]]
[[0, 1, 2, 3], [0, 1, 2, 5], [0, 1, 3, 4], [0, 1, 4, 5]]
Total number of excitations = 8


In [6]:
# VQE define the device, optimizer and circuit
import optax
opt = optax.sgd(learning_rate=0.8) # sgd stands for StochasticGradientDescent - increasing learning rate seems to help !
# initialize the gate parameters
# Note that starting from all zeros and hf qubit register init gives us a barren plateau and VQE failure
#init_params = jnp.zeros(len(singles) + len(doubles))
init_vals = np.random.rand(len(singles)+len(doubles))
init_params = jnp.array(init_vals)

@qml.qjit
def update_step(i, params, opt_state):
    """Perform a single gradient update step"""
    grads = qml.grad(circuit)(params)
    updates, opt_state = opt.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    return (params, opt_state)

opt_state = opt.init(init_params)
params = init_params
energy = 0.0
prev_energy = energy
for i in range(100):
    if (i % 10 == 0):
        print (i,params,energy)    
    params, opt_state = update_step(i, params, opt_state)
    energy = circuit(params)
    if jnp.abs(energy - prev_energy) < 1e-6:
        break
    prev_energy = energy
ground_state_energy = energy
ground_state_params = params
print ("ground state energy %s parameters %s " % (ground_state_energy, ground_state_params))

0 [0.05869696 0.706184   0.91819555 0.4433967  0.0732339  0.99163115
 0.6787446  0.36899516] 0.0
10 [0.19908297 0.17702639 1.1172736  0.24832663 0.43059897 0.26510888
 0.2214938  0.09001211] -147.56400682269614
20 [0.3267476  0.03647298 1.3328049  0.10822787 0.6734624  0.07509397
 0.05063926 0.04992758] -147.5937715607773
30 [0.47652704 0.01002512 1.5382162  0.05410179 0.8821155  0.03367584
 0.01352903 0.03800919] -147.60816610242736
40 [0.6224871  0.00378034 1.716684   0.03206262 1.0624197  0.02115015
 0.00473704 0.02938116] -147.61891496889314
50 [7.4247611e-01 1.8504320e-03 1.8542883e+00 2.1801885e-02 1.2064813e+00
 1.5795067e-02 2.2032412e-03 2.2224993e-02] -147.62565051608559
60 [8.30480635e-01 1.09673385e-03 1.95131564e+00 1.62548702e-02
 1.31420982e+00 1.28169125e-02 1.28735369e-03 1.64549593e-02] -147.62922943268302
70 [8.9100009e-01 7.4091071e-04 2.0163126e+00 1.2841704e-02 1.3916434e+00
 1.0823568e-02 8.7819784e-04 1.1977539e-02] -147.6309573518433
80 [9.3127507e-01 5.4589083

In [7]:
# Verify spin state
S2 = qml.qchem.spin2(active_electrons, qubits)
@qml.qnode(dev, interface="jax")
def S2_exp_value(parameters):
    AllSinglesDoubles(parameters, range(qubits), init_state, singles, doubles)
    return qml.expval(S2)

s2_val = S2_exp_value(ground_state_params)
print ('spin %s' % s2_val)

spin 1.9962171946438545


In [10]:
from functools import partial
# This line is added to better visualise the circuit with the optimised ground state parameters
@partial(qml.transforms.decompose, max_expansion=1)
def ansatz(parameters, wires):
    AllSinglesDoubles(parameters, range(qubits), init_state, singles, doubles)
    #return qml.expval(H)  # we are interested in minimizing this expectation value

#theta = np.random.rand(3) # 3 parameters for the ansatz
#theta=params
print(qml.draw(ansatz, decimals = 2)(ground_state_params,range(qubits)))

0: ─╭|Ψ⟩─╭G²(1.51)─╭G²(0.01)─╭G²(0.00)─╭G²(0.00)─╭G(0.97)─╭G(0.00)───────────────────┤  
1: ─├|Ψ⟩─├G²(1.51)─├G²(0.01)─├G²(0.00)─├G²(0.00)─│────────│────────╭G(2.10)─╭G(0.01)─┤  
2: ─├|Ψ⟩─├G²(1.51)─├G²(0.01)─│─────────│─────────╰G(0.97)─│────────│────────│────────┤  
3: ─├|Ψ⟩─╰G²(1.51)─│─────────├G²(0.00)─│──────────────────│────────╰G(2.10)─│────────┤  
4: ─├|Ψ⟩───────────│─────────╰G²(0.00)─├G²(0.00)──────────╰G(0.00)──────────│────────┤  
5: ─╰|Ψ⟩───────────╰G²(0.01)───────────╰G²(0.00)────────────────────────────╰G(0.01)─┤  


# First excited state optimisation

In [11]:
#from functools import partial
# Our trial state is defined with the same circuit but we assign random gate parameters
# The excitations are renumbered to match the wires position
# This line is added to better visualise the circuit
@partial(qml.transforms.decompose, max_expansion=1)

def ansatz(theta, wires):
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)#, delta_sz = 0 )
    #print(f"Total number of excitations = {len(singles) + len(doubles)}")
    #print (singles)
    singles = [[wires[i] for i in single] for single in singles]
    doubles = [[wires[i] for i in double] for double in doubles]
    #print (singles)
    AllSinglesDoubles(theta, wires, hf, singles, doubles)

theta = np.random.rand(8) # 8 parameters for the ansatz
print(qml.draw(ansatz, decimals = 2)(theta, range(7,13)))

 7: ─╭|Ψ⟩─╭G²(0.96)─╭G²(0.13)─╭G²(0.97)─╭G²(0.40)─╭G(0.25)─╭G(0.12)───────────────────┤  
 8: ─├|Ψ⟩─├G²(0.96)─├G²(0.13)─├G²(0.97)─├G²(0.40)─│────────│────────╭G(0.99)─╭G(0.11)─┤  
 9: ─├|Ψ⟩─├G²(0.96)─├G²(0.13)─│─────────│─────────╰G(0.25)─│────────│────────│────────┤  
10: ─├|Ψ⟩─╰G²(0.96)─│─────────├G²(0.97)─│──────────────────│────────╰G(0.99)─│────────┤  
11: ─├|Ψ⟩───────────│─────────╰G²(0.97)─├G²(0.40)──────────╰G(0.12)──────────│────────┤  
12: ─╰|Ψ⟩───────────╰G²(0.13)───────────╰G²(0.40)────────────────────────────╰G(0.11)─┤  


In [12]:
# To implement the SWAP test need to 
# lay on wires 1 to qubits+1 the ground state 
# lay on wires qubits+2 the ansatz to generate the excited state
# add Hadmard on wire 0
# add CSWAP for each pair of qubits
# add another Hadamard on wire 0
# measure Pauli Z operator on wire 0
dev = qml.device("lightning.qubit", wires=2*qubits+1)
@partial(qml.transforms.decompose, max_expansion=1)
@qml.qnode(dev)
def swap_test(params):
    # generate_ground_state(range(1, n_qubits + 1))
    #AllSinglesDoubles(ground_state_params, [1,2,3,4], hf, singles, doubles)
    ansatz(ground_state_params,range(1, qubits+1))
    ansatz(params, range(qubits + 1, 2 * qubits + 1))
    qml.Barrier()  # added to better visualise the circuit
    qml.Hadamard(wires=0)
    for i in range(qubits):
        qml.CSWAP(wires=[0, 1 + i + qubits, 1 + i])
    qml.Hadamard(wires=0)
    return qml.expval(qml.Z(0))

print(qml.draw(swap_test)(theta))
print(f"\nOverlap between the ground state and the ansatz: {swap_test(theta)}")

 0: ───────────────────────────────────────────────────────────────────────────────────||──H─╭●───
 1: ─╭|Ψ⟩─╭G²(1.51)─╭G²(0.01)─╭G²(0.00)─╭G²(0.00)─╭G(0.97)─╭G(0.00)────────────────────||────├SWAP
 2: ─├|Ψ⟩─├G²(1.51)─├G²(0.01)─├G²(0.00)─├G²(0.00)─│────────│────────╭G(2.10)─╭G(0.01)──||────│────
 3: ─├|Ψ⟩─├G²(1.51)─├G²(0.01)─│─────────│─────────╰G(0.97)─│────────│────────│─────────||────│────
 4: ─├|Ψ⟩─╰G²(1.51)─│─────────├G²(0.00)─│──────────────────│────────╰G(2.10)─│─────────||────│────
 5: ─├|Ψ⟩───────────│─────────╰G²(0.00)─├G²(0.00)──────────╰G(0.00)──────────│─────────||────│────
 6: ─╰|Ψ⟩───────────╰G²(0.01)───────────╰G²(0.00)────────────────────────────╰G(0.01)──||────│────
 7: ─╭|Ψ⟩─╭G²(0.96)─╭G²(0.13)─╭G²(0.97)─╭G²(0.40)─╭G(0.25)─╭G(0.12)────────────────────||────╰SWAP
 8: ─├|Ψ⟩─├G²(0.96)─├G²(0.13)─├G²(0.97)─├G²(0.40)─│────────│────────╭G(0.99)─╭G(0.11)──||─────────
 9: ─├|Ψ⟩─├G²(0.96)─├G²(0.13)─│─────────│─────────╰G(0.25)─│────────│────────│─────────||─────────
10: ─├|Ψ⟩─

In [13]:
# loss function
@qml.qnode(dev)
def expected_value(theta):
    ansatz(theta, range(qubits))
    return qml.expval(H)

def loss_f(theta, beta):
    return expected_value(theta) + beta * swap_test(theta)


In [14]:
# now optimise
import jax
import optax

jax.config.update("jax_platform_name", "cpu")
jax.config.update("jax_enable_x64", True)
print()

theta = jax.numpy.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])
beta = 2

# Store the values of the cost function
energy = [loss_f(theta, beta)]

conv_tol = 1e-6
max_iterations = 100

opt = optax.sgd(learning_rate=0.4)

# Store the values of the circuit parameter
angle = [theta]

opt_state = opt.init(theta)

for n in range(max_iterations):
    gradient = jax.grad(loss_f)(theta, beta)
    updates, opt_state = opt.update(gradient, opt_state)
    theta = optax.apply_updates(theta, updates)
    angle.append(theta)
    energy.append(loss_f(theta, beta))

    conv = jax.numpy.abs(energy[-1] - energy[-2])

    if n % 5 == 0:
        print(f"Step = {n},  Energy = {energy[-1]:.8f} Ha")

    if conv <= conv_tol:
        break

print(f"\nEstimated energy: {energy[-1].real:.8f}")
print(f"\nOptimised Parameters: {theta}")


Step = 0,  Energy = -147.37291532 Ha
Step = 5,  Energy = -147.50584032 Ha
Step = 10,  Energy = -147.54291399 Ha
Step = 15,  Energy = -147.55801559 Ha
Step = 20,  Energy = -147.56497864 Ha
Step = 25,  Energy = -147.56851171 Ha
Step = 30,  Energy = -147.57047390 Ha
Step = 35,  Energy = -147.57168414 Ha
Step = 40,  Energy = -147.57251954 Ha
Step = 45,  Energy = -147.57315711 Ha
Step = 50,  Energy = -147.57368136 Ha
Step = 55,  Energy = -147.57413357 Ha
Step = 60,  Energy = -147.57453465 Ha
Step = 65,  Energy = -147.57489581 Ha
Step = 70,  Energy = -147.57522356 Ha
Step = 75,  Energy = -147.57552212 Ha
Step = 80,  Energy = -147.57579453 Ha
Step = 85,  Energy = -147.57604318 Ha
Step = 90,  Energy = -147.57627014 Ha
Step = 95,  Energy = -147.57647720 Ha

Estimated energy: -147.57662964

Optimised Parameters: [-1.73314983e-01 -8.31615726e-06  9.72010063e-02  4.49307998e-04
  1.17457002e+00  8.87066246e-04  8.98467494e-04  2.77664168e-02]


In [15]:
# Verify the expectation values
#energy = circuit(theta)
s2_val = S2_exp_value(theta)
print (s2_val)

0.0027833763732739384


In [52]:
# Compare against eigenvalues from exact diagonalisation
print(np.sort(np.linalg.eigvals(H.matrix())))


[-147.63241465+0.j -147.63241465+0.j -147.63241465+0.j -147.57848667+0.j
 -147.57848667+0.j -147.52790164+0.j -147.3677911 +0.j -147.3677911 +0.j
 -147.3677911 +0.j -147.3677911 +0.j -147.21867808+0.j -147.21867808+0.j
 -147.21867808+0.j -147.21867808+0.j -147.15563792+0.j -147.15563792+0.j
 -147.15563792+0.j -147.15563792+0.j -147.15563792+0.j -147.15563792+0.j
 -147.07454873+0.j -147.07454873+0.j -146.95437927+0.j -146.95437927+0.j
 -146.95437927+0.j -146.95437927+0.j -146.8599067 +0.j -146.8599067 +0.j
 -146.8599067 +0.j -146.8599067 +0.j -146.83274548+0.j -146.83274548+0.j
 -146.80597872+0.j -146.80597872+0.j -146.78490902+0.j -146.78490902+0.j
 -146.54107526+0.j -146.53087339+0.j -146.2581239 +0.j -146.2581239 +0.j
 -146.2581239 +0.j -146.2581239 +0.j -146.23295397+0.j -146.03273253+0.j
 -146.03273253+0.j -146.03273253+0.j -146.03273253+0.j -146.03273253+0.j
 -146.03273253+0.j -145.95164334+0.j -145.95164334+0.j -145.40280882+0.j
 -145.40280882+0.j -145.40280882+0.j -145.34888085+